# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: ce98b428-9c04-4dfc-a7a2-a6d515a0d117
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can serve as a stepping stone to larger projects.


## Evaluation setup using TruLens

In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Context Relevance,Groundedness,Answer Relevance,Context Relevance_calls,Groundedness_calls,Answer Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_106dc937a432c46fe4a7247c5d7f7136,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_106dc937a432c46fe4a...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-08-01T05:06:23.597881"", ""...",2025-08-01T05:06:24.758765,1.00,1.000000,1.0,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",[{'args': {'prompt': 'What are the keys to bui...,1,2066,0.003123
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_827c6f243a48cf8be47b4a246e7736b7,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_827c6f243a48cf8be47...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-08-01T05:06:24.877500"", ""...",2025-08-01T05:06:26.579827,0.50,1.000000,1.0,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'How can teamwork contrib...,1,1693,0.002573
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2edcd080a7824859d46933b24855d211,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it helps indiv...",-,"{""record_id"": ""record_hash_2edcd080a7824859d46...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-08-01T05:06:26.694470"", ""...",2025-08-01T05:06:27.959962,0.10,0.533333,1.0,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'What is the importance o...,1,1693,0.002573
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_df616d5c9c5359454c2953f02bc45c6c,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_df616d5c9c5359454c2...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-08-01T05:06:28.072961"", ""...",2025-08-01T05:06:28.803173,1.00,1.000000,1.0,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'What are some good habit...,0,1631,0.002465
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3a2d876de64b47f84bfde56b822fc63f,"""How can altruism be beneficial in building a ...","""Helping others during your career journey can...",-,"{""record_id"": ""record_hash_3a2d876de64b47f84bf...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-08-01T05:06:28.909444"", ""...",2025-08-01T05:06:29.732645,0.55,0.875000,1.0,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,[{'args': {'prompt': 'How can altruism be bene...,0,1609,0.002421


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-29-22-122p38560.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [19]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [20]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

You can begin a personal project in AI by first identifying a project that aligns with your career goals. Ensure that the project is responsible, ethical, and beneficial to people. Once you have selected a project, focus on scoping it effectively by considering its potential impact and complexity. As you progress, aim to build a portfolio of projects that demonstrate your skill progression over time. This approach will not only help you develop your AI expertise but also enhance your career prospects in the field.


In [21]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [22]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.
How can teamwork contribute to success in AI?
Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on using AI to benefit all people can positively influence one's own work ethic and outcomes. The ability to work effectively in a team, leveraging each member's strengths and insights, can lead to improved project outcomes and overall success in AI endeavors.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and the lates

In [23]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.51,0.988889,0.371905,1.272727,0.000824


In [24]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-22-122p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [27]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 06582dba-3532-465f-bd1a-f780fd6f1b11.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: adbd1a2a-f8a1-4f5f-8f80-bdb869f17811.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

Building a portfolio of AI projects involves showcasing a progression from simple to complex undertakings over time. It is important to be able to communicate effectively about your work to demonstrate its value and gain trust from others. Identifying ideas that are worth working on is a crucial skill for an AI architect, and working on projects across various industries can help in gaining experience and diversifying your portfolio.


In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fb75344da80 is calling an instrumented method <function BaseQueryEngine.query at 0x7fb8be617640>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fb7d83aa3b0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fb75344da80 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fb8b948ea70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fb7d83aa3b0) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7fb7d82902e0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7fb8be6169e0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: a527ef67-deb0-4051-a352-2d1f257e6ef0.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: 8324eaad-219a-4da0-9530-da6c9f5e1339.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fb75344d9f0 is calling an instrumented method <function CompactAndRefine.get_response at 0x7fb8bdc3b520>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fb7d83ab5e0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fb75344d9f0 is calling an instrumented method <function Refine.get_response at 0x7fb8bce532e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fb7d83ab5e0) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7fb7d82093c0 is calling an instrumented method <function LLMPredictor.predict at 0x7fb8cab05120>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a community are key steps to building a career in AI. Additionally, collaborating with others, influencing, and being influenced by others are critical for success in tackling large projects in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fb75344d9f0 is calling an instrumented method <function Refine.get_response at 0x7fb8bce532e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fb7d83ab5e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fb75344da80 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fb8b948ea70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fb7d83aa3b0) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects, difficulties in project management due to uncertainty in time estimates for achieving target accuracy, and technical challenges that researchers and professionals in the field often face.
> Merging 3 nodes into parent node.
> Parent node id: dfa1bbec-0c46-44cd-bfc8-3bcc7765dd7c.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: aa471479-c1c2-4b2d-8a9d-0b68a4ca2346.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fb75344d9f0 is calling an instrumented method <function Refine.get_response at 0x7fb8bce532e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fb7d83ab5e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fb75344da80 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fb8b948ea70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fb7d83aa3b0) using this function.


Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.
> Merging 1 nodes into parent node.
> Parent node id: 67e65e2a-a55b-4c48-a4d9-5bc010704b8b.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 554b74c2-9360-43ac-a55e-0b70a6e60aad.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 1002df62-38aa-493a-857b-dd52393a89b6.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent node id: b56cd880-8af1-4678-818f-caa862b260dc.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 7abecc81-12c7-4ad3-a679-361d5c8182d1.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fb75344d9f0 is calling an instrumented method <function Refine.get_response at 0x7fb8bce532e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fb7d83ab5e0) using this function.


What is the right AI job for me?
The right AI job for you may depend on whether you are looking to switch roles, industries, or both. If you are seeking your first job in AI, transitioning either roles or industries individually may be more manageable than attempting both simultaneously.


In [30]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.708333,1.0,0.843056,1.909091,0.000861


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-22-122p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>